In [25]:
import pandas as pd
import numpy as np
import yfinance as yf
from statsmodels.api import OLS, add_constant
import plotly.graph_objects as go

def backtest_hedged_portfolio(portfolio_tickers, benchmark_ticker, start_date='2015-01-01', end_date='2025-01-01', window_size=252, rebalance_freq=120):
    # Load historical data
    data = yf.download(portfolio_tickers + [benchmark_ticker], start=start_date, end=end_date)['Close']
    
    # Compute daily returns
    returns = data.pct_change().dropna()
    
    # Create an equally weighted portfolio
    portfolio_returns = returns[portfolio_tickers].mean(axis=1)
    
    # Store portfolio returns
    portfolio_data = pd.DataFrame({'Portfolio': portfolio_returns, 'Benchmark': returns[benchmark_ticker]})
    
    # Initialize hedge ratio storage
    hedge_ratios = pd.Series(index=portfolio_data.index, dtype=float)
    
    # Rolling beta estimation and hedging
    for i in range(window_size, len(portfolio_data), rebalance_freq):
        # Define rolling window
        window_data = portfolio_data.iloc[i-window_size:i]
        
        # Estimate beta using OLS regression
        X = add_constant(window_data['Benchmark'])
        y = window_data['Portfolio']
        model = OLS(y, X).fit()
        beta = model.params['Benchmark']
        
        # Store hedge ratio (negative of beta for zero beta portfolio)
        hedge_ratios.iloc[i:i+rebalance_freq] = -beta
    
    # Forward fill last estimated beta
    hedge_ratios.ffill(inplace=True)
    
    # Compute hedged portfolio returns
    portfolio_data['Hedged'] = portfolio_data['Portfolio'] + hedge_ratios * portfolio_data['Benchmark']
    
    # Compute cumulative returns
    portfolio_data['Cumulative_Portfolio'] = (1 + portfolio_data['Portfolio']).cumprod()
    portfolio_data['Cumulative_Hedged'] = (1 + portfolio_data['Hedged']).cumprod()

    # Compute Hedge Effectiveness
    portfolio_data['HE'] = 1 - portfolio_data['Hedged'].rolling(252).var() / portfolio_data['Portfolio'].rolling(252).var() 

    
    return portfolio_data

# Example usage
portfolio_tickers = ['AAPL', 'MSFT', 'NVDA', 'AMZN', 'META', 'TSLA', 'GOOGL']
utilities_tickers = ['AES', 'LNT', 'AEE', 'AEP', 'AWK', 'ATO', 'CNP']
benchmark_ticker = 'SPY'
mag7_portfolio = backtest_hedged_portfolio(portfolio_tickers, benchmark_ticker, window_size=252, rebalance_freq=120)
utilities_portfolio = backtest_hedged_portfolio(utilities_tickers, benchmark_ticker, window_size=252, rebalance_freq=120)

[*********************100%***********************]  8 of 8 completed
[*********************100%***********************]  8 of 8 completed


In [26]:
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x = mag7_portfolio.index,
        y = mag7_portfolio['HE'],
        name = 'Hedge Effectiveness'
    )
)

fig.add_trace(
    go.Scatter(
        x = utilities_portfolio.index,
        y = utilities_portfolio['HE'],
        name = 'Hedge Effectiveness'
    )
)



fig.show()